<a href="https://colab.research.google.com/github/amylynnn/weatherwise-Amylynn-Sophie/blob/main/AmylynnSophieWeatherWiseApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
from datetime import datetime, timedelta
from random import choice

API_KEY = '7cf20335110caaf78db0fecb31852d45'
weather_cache = {}
CACHE_EXPIRY_MINUTES = 10

def fetch_weather_data(city, source='openweathermap'):
    global weather_cache
    key = (source.lower(), city.lower())
    now = datetime.utcnow()
    if key in weather_cache:
        timestamp, cached_data = weather_cache[key]
        if now - timestamp < timedelta(minutes=CACHE_EXPIRY_MINUTES):
            return cached_data
    if source.lower() == 'openweathermap':
        data = fetch_openweathermap(city)
    elif source.lower() == 'mocksource':
        data = fetch_mock_source(city)
    else:
        raise ValueError(f"Unsupported weather data source: {source}")
    weather_cache[key] = (now, data)
    return data

def fetch_openweathermap(city):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def fetch_mock_source(city):
    from random import uniform, randint
    now = datetime.utcnow()
    list_data = []
    for i in range(40):
        dt = now + timedelta(hours=3*i)
        dt_txt = dt.strftime("%Y-%m-%d %H:%M:%S")
        temp = round(uniform(10, 25), 1)
        rain = round(uniform(0, 5), 1) if randint(0, 1) else 0
        snow = round(uniform(0, 2), 1) if randint(0, 3) == 0 else 0
        weather_desc = 'clear sky' if randint(0, 2) == 0 else 'rain'
        entry = {
            'dt_txt': dt_txt,
            'main': {'temp': temp},
            'rain': {'3h': rain} if rain > 0 else {},
            'snow': {'3h': snow} if snow > 0 else {},
            'weather': [{'description': weather_desc}],
            'wind': {'speed': round(uniform(0, 10),1), 'deg': randint(0,360)}
        }
        list_data.append(entry)
    return {'list': list_data}

def parse_weather_question(question):
    question_lower = question.lower()

    condition_keywords = {
        'rain': ['rain', 'raining', 'rainy', 'wet', 'shower', 'showers', 'drizzle'],
        'snow': ['snow', 'snowing', 'snowy', 'flurries'],
        'clear': ['clear', 'sunny', 'bright'],
        'cloudy': ['cloud', 'cloudy', 'overcast'],
        'temperature': ['temperature', 'hot', 'cold', 'warm', 'chilly', 'freezing', 'heat'],
        'wind': ['wind', 'windy', 'breeze', 'gust'],
        'humidity': ['humidity', 'humid', 'dry'],
        'storm': ['storm', 'thunderstorm', 'lightning', 'hail'],
        'fog': ['fog', 'mist', 'haze']
    }

    condition = None
    for cond, synonyms in condition_keywords.items():
        for syn in synonyms:
            if syn in question_lower:
                condition = cond
                break
        if condition:
            break
    if not condition:
        condition = 'general'

    if 'day after tomorrow' in question_lower:
        time_frame = 'day_after_tomorrow'
    elif 'tomorrow' in question_lower:
        time_frame = 'tomorrow'
    elif 'today' in question_lower:
        time_frame = 'today'
    else:
        match = re.search(r'next (\d+) days', question_lower)
        if match:
            time_frame = f"next_{match.group(1)}_days"
        elif 'this weekend' in question_lower:
            time_frame = 'this_weekend'
        else:
            time_frame = 'today'

    # --- FIXED CITY EXTRACTION LOGIC ---
    city_match = re.search(
        r"in\s+([a-zA-ZÀ-ÿ\s\-]+?)(?:\s+(?:today|tomorrow|day after tomorrow|next \d+ days|this weekend)|\?|$| will| is| does| rain| snow| clear| cloudy| thunderstorm| drizzle| fog| mist| temperature| wind| humidity| storm)",
        question_lower)
    if city_match:
        city = city_match.group(1).strip()
    else:
        # fallback: extract last capitalized word(s) from original question
        capitalized = re.findall(r"\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)*", question)
        city = capitalized[-1] if capitalized else None

    # Extra robustness: remove leading 'in ' if still present
    if city and city.lower().startswith("in "):
        city = city[3:].strip()

    fixes = {
        "paris": "Paris,FR",
        "london": "London,GB",
        "new york": "New York,US",
        "berlin": "Berlin,DE",
        "sydney": "Sydney,AU"
    }
    if city:
        city_key = city.lower()
        city = fixes.get(city_key, city.title())
    else:
        city = None

    return {
        'condition': condition,
        'city': city,
        'time_frame': time_frame
    }

def generate_weather_response(parsed, source='openweathermap'):
    if not parsed or not parsed.get('city'):
        return "⚠️ Sorry, I couldn't understand your question or detect the city. Please try again."

    condition = parsed['condition']
    city = parsed['city']
    time_frame = parsed['time_frame']

    try:
        data = fetch_weather_data(city, source=source)
        today = datetime.utcnow().date()

        if time_frame == 'tomorrow':
            target_date = today + timedelta(days=1)
        elif time_frame == 'day_after_tomorrow':
            target_date = today + timedelta(days=2)
        elif time_frame.startswith('next_'):
            days = int(time_frame.split('_')[1])
            temps = []
            rains = []
            for entry in data['list']:
                entry_date = datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S").date()
                if today <= entry_date < today + timedelta(days=days):
                    temps.append(entry['main']['temp'])
                    rains.append(entry.get('rain', {}).get('3h', 0))
            if temps:
                avg_temp = sum(temps) / len(temps)
                total_rain = sum(rains)
                return (f"🌤️ Over the next {days} days in {city}, expect an average temperature of "
                        f"{avg_temp:.1f}°C and total precipitation around {total_rain:.1f} mm.")
            else:
                return f"⚠️ Sorry, no forecast data available for the next {days} days in {city}."
        else:
            target_date = today

        day_entries = [entry for entry in data['list'] if datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S").date() == target_date]
        if not day_entries:
            return f"⚠️ Sorry, no forecast data available for {time_frame.replace('_', ' ')} in {city}."

        temps = [entry['main']['temp'] for entry in day_entries]
        avg_temp = sum(temps) / len(temps)
        rain_volumes = [entry.get('rain', {}).get('3h', 0) for entry in day_entries]
        total_rain = sum(rain_volumes)
        wind_speeds = [entry.get('wind', {}).get('speed', 0) for entry in day_entries]
        max_wind = max(wind_speeds) if wind_speeds else 0

        will_rain = total_rain > 1
        is_cold = avg_temp < 10
        is_hot = avg_temp > 25
        windy = max_wind > 15

        messages = []
        messages.append(f"🌤️ Weather forecast for {city} {time_frame.replace('_', ' ')}:")
        messages.append(f"Average temperature: {avg_temp:.1f}°C.")

        if will_rain:
            rain_msgs = [
                "Pack an umbrella, unless you're secretly a duck. ☔",
                "Rain is on the forecast! Time to dance in the puddles...or just stay inside with a good book. 🌧️",
                "The sky is crying today - bring a raincoat and cheer it up! 😢",
            ]
            messages.append(choice(rain_msgs))
        else:
            clear_msgs = [
                "Clear skies ahead! Time to wear those sunglasses 😎",
                "Looks like the sun's got its hat on! Don't forget the sunscreen. ☀️",
                "Not a cloud in sight - perfect day for a picnic! 🧺",
            ]
            messages.append(choice(clear_msgs))

        if is_cold:
            cold_msgs = [
                "It's colder than a penguin's toes out there! Bundle up! 🐧",
                "Time to build a snowman... or just hide under a blanket. ☃️",
                "Beware of rogue icicles! Dress warmly, or you'll turn into a human popsicle. 🥶",
            ]
            messages.append(choice(cold_msgs))
        elif is_hot:
            hot_msgs = [
                "It's hotter than a dragon's breath! Stay hydrated! 🐉",
                "Time to hit the beach...or just sit in front of the AC. 🏖️",
                "Make sure to hydrate today, or you may end up melting like the polar ice caps. 🥵",
            ]
            messages.append(choice(hot_msgs))

        if windy:
            windy_msgs = [
                "Hold onto your hats, it's gonna be a wild one! 💨",
                "Prepare for a hair-raising experience - the wind is howling! 💇‍♀️",
                "Make sure all loose items are secured, or they may end up in another country! 🌪️",
            ]
            messages.append(choice(windy_msgs))

        messages.append(f"Total precipitation expected: {total_rain:.1f} mm.")
        messages.append("Have a great day! ☀️")

        return "\n".join(messages)

    except Exception as e:
        return f"❌ Could not retrieve weather for {city}: {e}"

def interactive_temperature_plot(data, city):
    times = [datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S") for entry in data['list']]
    temps = [entry['main']['temp'] for entry in data['list']]

    def plot_upto(index=8):
        plt.figure(figsize=(12, 6))
        plt.plot(times[:index], temps[:index], marker='o', linestyle='-', color='orange')
        plt.title(f"Temperature Forecast for {city.title()}")
        plt.xlabel("Date & Time")
        plt.ylabel("Temperature (°C)")
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    widgets.interact(plot_upto, index=widgets.IntSlider(min=1, max=len(times), step=1, value=8))

def interactive_precipitation_plot(data, city):
    times = [datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S") for entry in data['list']]
    rain = [entry.get('rain', {}).get('3h', 0) for entry in data['list']]

    def plot_upto(index=8):
        plt.figure(figsize=(12, 6))
        plt.bar(times[:index], rain[:index], width=0.1, color='blue', edgecolor='black')
        plt.title(f"Precipitation Forecast for {city.title()} (Next 5 Days)")
        plt.xlabel("Date & Time")
        plt.ylabel("Rain (mm in 3h)")
        plt.xticks(rotation=45)
        plt.grid(axis='y', linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.show()

    widgets.interact(plot_upto, index=widgets.IntSlider(min=1, max=len(times), step=1, value=8))

def interactive_ui():
    city_input = widgets.Text(value='', placeholder='Enter city (e.g. Paris)', description='City:', layout=widgets.Layout(width='300px'))
    question_input = widgets.Text(value='', placeholder='e.g. Will it rain in Paris tomorrow?', description='Ask:', layout=widgets.Layout(width='500px'))
    source_dropdown = widgets.Dropdown(options=['openweathermap', 'mocksource'], value='openweathermap', description='Source:', layout=widgets.Layout(width='200px'))

    output_area = widgets.Output(layout=widgets.Layout(
    border='1px solid #e0e0e0',
    padding='22px',
    max_height='440px',
    overflow='auto',
    background_color='#f7fafc',
    border_radius='12px'
))


    forecast_btn = widgets.Button(description="📊 Forecast Charts", button_style='info', tooltip='Show temperature and precipitation charts')
    question_btn = widgets.Button(description="🧠 Answer", button_style='success', tooltip='Get AI-style weather answer')
    clear_btn = widgets.Button(description="🧹 Clear", button_style='warning', tooltip='Clear inputs and output')

    def toggle_buttons(*args):
        forecast_btn.disabled = not bool(city_input.value.strip())
        question_btn.disabled = not bool(question_input.value.strip())

    city_input.observe(toggle_buttons, 'value')
    question_input.observe(toggle_buttons, 'value')
    toggle_buttons()

    def on_forecast_button_click(_):
        with output_area:
            clear_output()
            city = city_input.value.strip()
            if not city:
                print("❌ Please enter a city.")
                return
            try:
                data = fetch_weather_data(city, source=source_dropdown.value)
                interactive_temperature_plot(data, city)
                interactive_precipitation_plot(data, city)
            except Exception as e:
                print(f"❌ Error: {e}")

    def on_question_button_click(_):
        with output_area:
            clear_output()
            question = question_input.value.strip()
            if not question:
                print("❌ Please enter a question.")
                return
            parsed = parse_weather_question(question)
            response = generate_weather_response(parsed, source=source_dropdown.value)
            print(response)

    def on_clear_button_click(_):
        city_input.value = ''
        question_input.value = ''
        with output_area:
            clear_output()

    forecast_btn.on_click(on_forecast_button_click)
    question_btn.on_click(on_question_button_click)
    clear_btn.on_click(on_clear_button_click)

    input_row = widgets.HBox([city_input, source_dropdown, forecast_btn, clear_btn], layout=widgets.Layout(margin='10px 0'))
    question_row = widgets.HBox([question_input, question_btn], layout=widgets.Layout(margin='10px 0'))

    header = widgets.HTML("""

        🌤️ Interactive Weather Assistant

""")


    main_box = widgets.VBox([
    header,
    input_row,
    question_row,
    output_area
], layout=widgets.Layout(
    width='820px',
    max_width='96vw',
    margin='48px auto 48px auto',
    padding='36px 48px 40px 48px',
    background_color='#fff',
    border_radius='26px',
    border='1.5px solid #e3e8ee',
    box_shadow='0 8px 32px rgba(44, 62, 80, 0.13)',
    display='flex',
    flex_flow='column',
    align_items='center',
    justify_content='center',
    font_family='Segoe UI, Arial, sans-serif'
))


    display(main_box)

interactive_ui()